In [1]:
# Import libraries
import pandas as pd
import numpy as np
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS

import matplotlib.pyplot as plt
import logging
import os
import sys

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable
os.environ["HADOOP_HOME"] = ""
os.environ["hadoop.home.dir"] = ""

logging.getLogger().setLevel(logging.INFO)

# Set plot parameters
plt.rcParams["figure.figsize"] = (20, 13)
%matplotlib inline
%config InlineBackend.figure_format = "retina"

Concernant la méthode abordé pour le projet final, j'ai décidé de prendre comme méthode la méthode ALS (Alternating Least Squares) appliqué en cours.

Dans l'idée, comme cité dans le cours, l’algorithme ALS (Alternating Least Squares) vise à minimiser la même fonction objective d’erreur au carré que FunkSVD, mais aborde le problème de minimisation en utilisant une méthode de moindres carrés alternée.
L’algorithme fonctionne en fixant d’abord la matrice user et en minimisant la fonction objective par rapport à la matrice item(dans notre cas vendor), puis en fixant la matrice item et en minimisant la fonction objective par rapport à la matrice user. Ce processus est répété jusqu’à convergence. À chaque itération, l’algorithme résout un problème de moindres carrés pour mettre à jour la matrice user ou item.

De ce fait, il a fallut que je fasse en premier lieu un preprocessing des données pour pouvoir appliquer la méthode qui m'était convenu.

# Preprocessing Train

## Analyse des données Train Customers
Dans un premier temps, j'ai décidé de faire une analyse des données pour pouvoir comprendre les données que j'avais à disposition.
En effet, c'est un point important pour pouvoir faire un bon preprocessing des données et pouvoir discerner les données utiles des autres données mais aussi de pouvoir compléter les données manquantes.
Dans les données récupérées, j'ai décidé de garder les colonnes suivantes qui me semblaient utiles pour la suite :
- gender : comprend les valeurs M, F, ? et vide donc il faudra faire un traitement pour ne garder que M et F
- verified : comprend les valeurs 0 et 1 donc il faudra faire un traitement pour ne garder que 1
- created_at : comprend des dates donc il faudra faire un traitement pour pouvoir les utiliser
- updated_at : comprend des dates donc il faudra faire un traitement pour pouvoir les utiliser
- customer_id : comprend des identifiants donc il faudra faire un traitement pour pouvoir les utiliser

In [2]:
df_users_training = pd.read_csv('./data/train_customers.csv')

In [3]:
#df_users_training.describe(include="all")
#df_users_training.info(memory_usage="deep")
df_users_training["created_at"] = pd.to_datetime(df_users_training["created_at"])
df_users_training["updated_at"] = pd.to_datetime(df_users_training["updated_at"])
df_users_training.rename(columns={'akeed_customer_id': 'customer_id'}, inplace=True)

# Drop useless columns
df_users_training.drop(['language', 'dob', 'created_at'], axis=1, inplace=True)

#make gender only two values
df_users_training.gender.replace(to_replace=[r'[F|f].*', r'[M|m|?| ].*'], value=['F', 'M'], inplace=True, regex=True)

#take only verified users
df_users_training = df_users_training[df_users_training.verified == 1]

# drop verified column and status column
df_users_training.drop(['verified'], axis=1, inplace=True)
df_users_training.drop(['status'], axis=1, inplace=True)

## Analyse des données Train Locations
Dans un second temps, il m'a fallut faire une analyse des données de la table Locations pour pouvoir faire un preprocessing des données.
Le but étant de rejoindre toutes les données entre elles et de pouvoir faire des prédictions sur les données manquantes.
Dans les données récupérées, j'ai décidé de garder les colonnes suivantes qui me semblaient utiles pour la suite :
- location_number : comprend des identifiants donc il faudra faire un traitement pour pouvoir les utiliser
- location_type : comprend des valeurs comme Home, Work, Other, etc. donc il faudra faire un traitement pour pouvoir les utiliser à savoir faire en sorte que les NaN deviennent Other
- latitude : correspond à la latitude comme son nom l'indique
- longitude : correspond à la longitude comme son nom l'indique
- customer_id : comprend des identifiants qui permettra de faire la jointure avec la table Customers

In [4]:
df_users_loc_training = pd.read_csv('./data/train_locations.csv')
#df_users_loc_training.describe(include="all")
#df_users_loc_training.info(memory_usage="deep")
# fill missing values from location_type
df_users_loc_training.location_type.fillna('Other', inplace=True)

## Analyse des données Train Orders
Dans un troisième temps, il a été d'une importance d'analyser les datas d'order puisque la colonne de distance nous permettra de pouvoir faire un rating.
Bien entendu, il a fallut faire un preprocessing des données pour pouvoir les utiliser et le fusionner avec les autres tables par la suite.

In [5]:
df_orders = pd.read_csv('./data/orders.csv')
#df_orders.describe(include='all')
#df_orders.info(memory_usage="deep")
# lier avec customer_id, location_number et location_type
df_orders.promo_code=df_orders.promo_code.fillna('', inplace=True)
df_orders.promo_code=df_orders.promo_code.astype('bool')
df_orders.promo_code_discount_percentage.fillna(0, inplace=True)
df_orders.LOCATION_TYPE.fillna('Other', inplace=True)
df_orders.drop(['delivery_time'], axis=1, inplace=True)
df_orders.rename(columns={'LOCATION_TYPE': 'location_type'}, inplace=True)
df_orders.rename(columns={'LOCATION_NUMBER': 'location_number'}, inplace=True)
# Supprimer les lignes sans order_id
# promo code important ? ou présence d'un promo code important
# promo_code_discount_percentage > promo_code : promo_code_discount_percentage peut être à 0
df_orders.is_favorite.fillna('', inplace=True)
df_orders.is_favorite = df_orders.is_favorite.astype('bool')
df_orders.is_favorite = df_orders.is_favorite.replace({True: 1, False: 0})
df_orders.promo_code.fillna('', inplace=True)
df_orders.promo_code = df_orders.promo_code.astype('bool')
df_orders.promo_code = df_orders.promo_code.replace({True: 1, False: 0})
df_orders.is_rated = df_orders.is_rated.replace({'No': 0, 'Yes': 1})
df_orders.akeed_order_id = df_orders.akeed_order_id.dropna()

C:\Users\phi-h\AppData\Local\Temp\ipykernel_55356\1648026616.py:1: DtypeWarning: Columns (15,16,18,19,20) have mixed types. Specify dtype option on import or set low_memory=False.
  df_orders = pd.read_csv('./data/orders.csv')


In [6]:
df_users_training = pd.merge(df_users_training, df_users_loc_training, on='customer_id', how='left')
df_users_training = df_users_training.dropna()

In [7]:
df_merge_training = pd.merge(df_users_training, df_orders, on=['customer_id', 'location_number', 'location_type'])

Comme dis précédemment, nous utilisons la colonne de distance pour pouvoir faire un rating.
le rating est calculé de la manière suivante :
- Normaliser la distance entre 0 et 1
- Convertir la distance normalisée en note
- La note est comprise entre 1 et 5
- Plus la distance est grande, plus la note est petite
- Plus la distance est petite, plus la note est grande
- La note est inversement proportionnelle à la distance


In [8]:
def distance_to_rating(distance, min_distance, max_distance, min_rating, max_rating):
    # Normaliser la distance entre 0 et 1
    normalized_distance = (distance - min_distance) / (max_distance - min_distance)

    # Convertir la distance normalisée en note
    rating = normalized_distance * (max_rating - min_rating) + min_rating

    return rating

In [9]:
# Exemple d'utilisation
distances = np.array([1.0, 2.0, 3.0])
min_distance = 1.0
max_distance = 3.0
min_rating = 1.0 # Inverser l'ordre des notes
max_rating = 5.0 # Inverser l'ordre des notes
#create a column in df_merge_training with the rating
df_merge_training['rating'] = distance_to_rating(df_merge_training.deliverydistance, df_merge_training.deliverydistance.min(), df_merge_training.deliverydistance.max(), min_rating, max_rating)

In [10]:
df_matrix_train = df_merge_training[['customer_id', 'location_number', 'vendor_id', 'rating']]
df_matrix_train = df_matrix_train.assign(C=lambda x: x['customer_id'].astype(str) + ' X ' + x['location_number'].astype(str) + ' X ' + x['vendor_id'].astype(str))
df_matrix_train.rename(columns={'C': 'CID X LOC_NUM X VENDOR', 'vendor_id': 'vendor'}, inplace=True)
#remove duplicates
df_matrix_train = df_matrix_train.drop_duplicates(subset=['CID X LOC_NUM X VENDOR'])
df_matrix_train = df_matrix_train[['CID X LOC_NUM X VENDOR', 'vendor', 'rating']]
df_matrix_train = df_matrix_train.dropna()
df_matrix_train

,CID X LOC_NUM X VENDOR,vendor,rating
0,TCHWPBT X 0 X 237,237,1.000000
1,TCHWPBT X 2 X 113,113,1.000000
2,ZGFSYCZ X 0 X 303,303,1.000000
7,ZGFSYCZ X 0 X 274,274,1.000000
8,ZGFSYCZ X 1 X 33,33,1.000000
...,...,...,...
77850,U2OTA4O X 0 X 573,573,1.810783
77851,Z7RQ368 X 0 X 160,160,2.797823
77852,WIIU12E X 0 X 573,573,1.501814
77853,LE63M0S X 0 X 84,84,1.640747


# Conversion de df_train en dataframe spark pour l'ALS

In [11]:
spark = SparkSession.builder.appName("ALSMatrixFactorisation").config("spark.executor.memory", "16g").config("spark.driver.memory","16g").getOrCreate()
df_training = spark.createDataFrame(df_matrix_train)
df_training.show()

+----------------------+------+------------------+
|CID X LOC_NUM X VENDOR|vendor|            rating|
+----------------------+------+------------------+
|     TCHWPBT X 0 X 237|   237|               1.0|
|     TCHWPBT X 2 X 113|   113|               1.0|
|     ZGFSYCZ X 0 X 303|   303|               1.0|
|     ZGFSYCZ X 0 X 274|   274|               1.0|
|      ZGFSYCZ X 1 X 33|    33|               1.0|
|      ZGFSYCZ X 1 X 28|    28|               1.0|
|     ZGFSYCZ X 1 X 310|   310|               1.0|
|     ZGFSYCZ X 3 X 300|   300|               1.0|
|     ZGFSYCZ X 3 X 159|   159|               1.0|
|      ZGFSYCZ X 4 X 33|    33|               1.0|
|     ZGFSYCZ X 5 X 221|   221|               1.0|
|      ZGFSYCZ X 5 X 33|    33|               1.0|
|     ZGFSYCZ X 5 X 303|   303|               1.0|
|    ZGFSYCZ X 11 X 356|   356|               1.0|
|    ZGFSYCZ X 11 X 845|   845|               1.0|
|    ZGFSYCZ X 12 X 582|   582|1.1099015033696216|
|    ZGFSYCZ X 13 X 676|   676|

In [12]:
indexer = [StringIndexer(inputCol=column, outputCol=column+"_index") for column in list(set(df_training.columns)-set(['rating'])) ]
pipeline = Pipeline(stages=indexer)
df_training = pipeline.fit(df_training).transform(df_training)
df_training.show()

+----------------------+------+------------------+------------+----------------------------+
|CID X LOC_NUM X VENDOR|vendor|            rating|vendor_index|CID X LOC_NUM X VENDOR_index|
+----------------------+------+------------------+------------+----------------------------+
|     TCHWPBT X 0 X 237|   237|               1.0|        53.0|                     40582.0|
|     TCHWPBT X 2 X 113|   113|               1.0|         0.0|                     40583.0|
|     ZGFSYCZ X 0 X 303|   303|               1.0|        85.0|                     49174.0|
|     ZGFSYCZ X 0 X 274|   274|               1.0|        63.0|                     49173.0|
|      ZGFSYCZ X 1 X 33|    33|               1.0|        22.0|                     49177.0|
|      ZGFSYCZ X 1 X 28|    28|               1.0|        15.0|                     49175.0|
|     ZGFSYCZ X 1 X 310|   310|               1.0|        74.0|                     49176.0|
|     ZGFSYCZ X 3 X 300|   300|               1.0|        67.0|       

# Preprocessing Test

## Analyse des données Test Customers

In [13]:
df_users_test = pd.read_csv('./data/test_customers.csv')
df_users_test.describe(include="all")
df_users_test.info(memory_usage="deep")
df_users_test["created_at"] = pd.to_datetime(df_users_test["created_at"])
df_users_test["updated_at"] = pd.to_datetime(df_users_test["updated_at"])
df_users_test.rename(columns={'akeed_customer_id': 'customer_id'}, inplace=True)

# Drop useless columns
df_users_test.drop(['language', 'dob', 'created_at'], axis=1, inplace=True)

#make gender only two values
df_users_test.gender.replace(to_replace=[r'[F|f].*', r'[M|m|?| ].*'], value=['F', 'M'], inplace=True, regex=True)

#take only verified users
df_users_test = df_users_test[df_users_test.verified == 1]


# drop verified column and status column
#df_users_test.drop(['verified'], axis=1, inplace=True)
#df_users_test.drop(['status'], axis=1, inplace=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9768 entries, 0 to 9767
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   akeed_customer_id  9768 non-null   object 
 1   gender             6321 non-null   object 
 2   dob                848 non-null    float64
 3   status             9768 non-null   int64  
 4   verified           9768 non-null   int64  
 5   language           5928 non-null   object 
 6   created_at         9768 non-null   object 
 7   updated_at         9768 non-null   object 
dtypes: float64(1), int64(2), object(5)
memory usage: 3.2 MB


## Analyse des données Test Locations

In [14]:
df_users_loc_test = pd.read_csv('./data/test_locations.csv')
df_users_loc_test.describe(include="all")
df_users_loc_test.info(memory_usage="deep")
# fill missing values from location_type
df_users_loc_test.location_type.fillna('Other', inplace=True)
#df_users_loc_test.location_type.fillna('Other', inplace=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16720 entries, 0 to 16719
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   customer_id      16720 non-null  object 
 1   location_number  16720 non-null  int64  
 2   location_type    9070 non-null   object 
 3   latitude         16717 non-null  float64
 4   longitude        16717 non-null  float64
dtypes: float64(2), int64(1), object(2)
memory usage: 2.2 MB


In [15]:
df_users_test = pd.merge(df_users_test, df_users_loc_test, on='customer_id', how='right')
df_users_test.rename(columns={'customer_id_index': 'user_id','latitude':'latitude_user','longitude': 'longitude_user'}, inplace=True)
df_users_test.info(memory_usage="deep")
# get all values of the column gender
df_users_test['gender'].

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16720 entries, 0 to 16719
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   customer_id      16720 non-null  object        
 1   gender           11348 non-null  object        
 2   status           15814 non-null  float64       
 3   verified         15814 non-null  float64       
 4   updated_at       15814 non-null  datetime64[ns]
 5   location_number  16720 non-null  int64         
 6   location_type    16720 non-null  object        
 7   latitude_user    16717 non-null  float64       
 8   longitude_user   16717 non-null  float64       
dtypes: datetime64[ns](1), float64(4), int64(1), object(3)
memory usage: 3.6 MB


## Analyse des données Test Vendors

In [16]:
df_client = pd.read_csv('./data/vendors.csv')
df_client = df_client[['id', 'latitude', 'longitude']]
df_client.rename(columns={'id': 'vendor_id','latitude': 'latitude_vendor', 'longitude': 'longitude_vendor'}, inplace=True)
df_matrix_test = pd.merge(df_users_test, df_client, how='cross')
#show the first 20 rows
df_matrix_test.head(20)
df_matrix_test.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1672000 entries, 0 to 1671999
Data columns (total 12 columns):
 #   Column            Non-Null Count    Dtype         
---  ------            --------------    -----         
 0   customer_id       1672000 non-null  object        
 1   gender            1134800 non-null  object        
 2   status            1581400 non-null  float64       
 3   verified          1581400 non-null  float64       
 4   updated_at        1581400 non-null  datetime64[ns]
 5   location_number   1672000 non-null  int64         
 6   location_type     1672000 non-null  object        
 7   latitude_user     1671700 non-null  float64       
 8   longitude_user    1671700 non-null  float64       
 9   vendor_id         1672000 non-null  int64         
 10  latitude_vendor   1672000 non-null  float64       
 11  longitude_vendor  1672000 non-null  float64       
dtypes: datetime64[ns](1), float64(6), int64(2), object(3)
memory usage: 394.2 MB


Comme dit précédemment, nous allons utiliser la distance entre le client et le restaurant pour calculer la note de ce dernier. Nous allons donc calculer la distance entre le client et le restaurant via la latitude et la longitude de chacun.

In [17]:
#get deliverydistance and rating
def distance(lat1, lon1, lat2, lon2):
    # rayon de la terre en km
    R = 6371

    # conversion en radian
    lat1 = np.radians(lat1)
    lon1 = np.radians(lon1)
    lat2 = np.radians(lat2)
    lon2 = np.radians(lon2)

    # formule
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = np.sin(dlat / 2)**2 + np.cos(lat1) * \
        np.cos(lat2) * np.sin(dlon / 2)**2
    c = 2 * np.arcsin(np.sqrt(a))

    # distance en km
    distance = R * c

    return distance

In [18]:
df_matrix_test['deliverydistance'] = distance(df_matrix_test.latitude_user, df_matrix_test.longitude_user, df_matrix_test.latitude_vendor, df_matrix_test.longitude_vendor)
df_matrix_test['rating'] = distance_to_rating(df_matrix_test.deliverydistance, df_matrix_test.deliverydistance.min(), df_matrix_test.deliverydistance.max(), min_rating, max_rating)
df_matrix_test= df_matrix_test[['customer_id','location_number','vendor_id','rating']]
df_matrix_test = df_matrix_test.assign(C=lambda x: x['customer_id'].astype(str) + ' X ' + x['location_number'].astype(str) + ' X ' + x['vendor_id'].astype(str))
df_matrix_test.rename(columns={'C': 'CID X LOC_NUM X VENDOR', 'vendor_id': 'vendor'}, inplace=True)
#remove duplicates
df_matrix_test = df_matrix_test.drop_duplicates(subset=['CID X LOC_NUM X VENDOR'])
df_matrix_test = df_matrix_test[['CID X LOC_NUM X VENDOR', 'vendor', 'rating']]
df_matrix_test

,CID X LOC_NUM X VENDOR,vendor,rating
0,Z59FTQD X 0 X 4,4,4.051932
1,Z59FTQD X 0 X 13,13,4.049152
2,Z59FTQD X 0 X 20,20,4.047912
3,Z59FTQD X 0 X 23,23,4.051857
4,Z59FTQD X 0 X 28,28,4.026828
...,...,...,...
1671995,3O8LSR3 X 0 X 849,849,1.040217
1671996,3O8LSR3 X 0 X 855,855,1.056561
1671997,3O8LSR3 X 0 X 856,856,1.013522
1671998,3O8LSR3 X 0 X 858,858,1.007700


## Conversion de df_test en dataframe spark pour l'ALS

In [19]:
#spark_2 = SparkSession.builder.appName("ALSMatrixFactorisationTEST").config("spark.executor.memory", "16g").config("spark.driver.memory","16g").getOrCreate()
df_test = spark.createDataFrame(df_matrix_test)
df_test.show()

+----------------------+------+------------------+
|CID X LOC_NUM X VENDOR|vendor|            rating|
+----------------------+------+------------------+
|       Z59FTQD X 0 X 4|     4|4.0519319851434785|
|      Z59FTQD X 0 X 13|    13| 4.049151517493291|
|      Z59FTQD X 0 X 20|    20| 4.047911790961834|
|      Z59FTQD X 0 X 23|    23| 4.051856681483615|
|      Z59FTQD X 0 X 28|    28| 4.026827735173355|
|      Z59FTQD X 0 X 33|    33| 4.049707754461085|
|      Z59FTQD X 0 X 43|    43| 4.041168522886974|
|      Z59FTQD X 0 X 44|    44|   4.0622632429566|
|      Z59FTQD X 0 X 55|    55| 4.067821284898708|
|      Z59FTQD X 0 X 66|    66| 4.026626544646668|
|      Z59FTQD X 0 X 67|    67|4.0429185235927925|
|      Z59FTQD X 0 X 75|    75| 4.052211766950357|
|      Z59FTQD X 0 X 76|    76| 4.032150332283219|
|      Z59FTQD X 0 X 78|    78| 4.052770278378749|
|      Z59FTQD X 0 X 79|    79|4.0231979637790705|
|      Z59FTQD X 0 X 81|    81| 4.056912698737247|
|      Z59FTQD X 0 X 82|    82|

In [20]:
indexer = [StringIndexer(inputCol=column, outputCol=column+"_index") for column in list(set(df_test.columns)-set(['rating'])) ]
pipeline = Pipeline(stages=indexer)
df_test = pipeline.fit(df_test).transform(df_test)
df_test.show()

+----------------------+------+------------------+------------+----------------------------+
|CID X LOC_NUM X VENDOR|vendor|            rating|vendor_index|CID X LOC_NUM X VENDOR_index|
+----------------------+------+------------------+------------+----------------------------+
|       Z59FTQD X 0 X 4|     4|4.0519319851434785|        59.0|                   1631859.0|
|      Z59FTQD X 0 X 13|    13| 4.049151517493291|         6.0|                   1631806.0|
|      Z59FTQD X 0 X 20|    20| 4.047911790961834|        27.0|                   1631827.0|
|      Z59FTQD X 0 X 23|    23| 4.051856681483615|        34.0|                   1631834.0|
|      Z59FTQD X 0 X 28|    28| 4.026827735173355|        43.0|                   1631843.0|
|      Z59FTQD X 0 X 33|    33| 4.049707754461085|        54.0|                   1631854.0|
|      Z59FTQD X 0 X 43|    43| 4.041168522886974|        62.0|                   1631862.0|
|      Z59FTQD X 0 X 44|    44|   4.0622632429566|        63.0|       

# Création du modèle ALS

In [21]:
als = ALS(
    maxIter=5,
    regParam=0.09,
    rank=25,
    userCol="CID X LOC_NUM X VENDOR_index",
    itemCol="vendor_index",
    ratingCol="rating",
    coldStartStrategy="drop",
    nonnegative=True,
)
model = als.fit(df_training)

# Evaluation du modèle

In [22]:
evaluator = RegressionEvaluator(
    metricName="rmse", labelCol="rating", predictionCol="prediction"
)

In [23]:
predictions = model.transform(df_test)
predictions = predictions.na.fill({'prediction': 0})
rmse = evaluator.evaluate(predictions)

print("RMSE=" + str(rmse))

RMSE=1.631113253552553


# Recommandation pour tous les utilisateurs

In [24]:
# Generate top 20 vendor recommendations for each user
userRecs = model.recommendForAllUsers(20).show()

+----------------------------+--------------------+
|CID X LOC_NUM X VENDOR_index|     recommendations|
+----------------------------+--------------------+
|                          31|[{90, 1.4472833},...|
|                          34|[{5, 2.9394953}, ...|
|                          53|[{34, 2.1476676},...|
|                          65|[{62, 2.7835677},...|
|                          78|[{4, 2.6957803}, ...|
|                          85|[{6, 2.6413693}, ...|
|                         108|[{6, 1.2538116}, ...|
|                         133|[{80, 0.9401024},...|
|                         137|[{23, 0.89890945}...|
|                         148|[{91, 2.1176667},...|
|                         155|[{4, 2.348916}, {...|
|                         193|[{15, 0.90692633}...|
|                         211|[{86, 2.1137369},...|
|                         243|[{75, 0.9059224},...|
|                         251|[{99, 2.1829667},...|
|                         255|[{61, 0.89535207}...|
|           

# Résultats

In [25]:
# get f1 score
def get_scores(df):
    df = df.withColumn("TP", when((col("rating") >= 4) & (col("prediction") >= 4), 1).otherwise(0))
    df = df.withColumn("FP", when((col("rating") < 4) & (col("prediction") >= 4), 1).otherwise(0))
    df = df.withColumn("FN", when((col("rating") >= 4) & (col("prediction") < 4), 1).otherwise(0))
    df = df.withColumn("TN", when((col("rating") < 4) & (col("prediction") < 4), 1).otherwise(0))
    TP = df.agg(sum("TP")).collect()[0][0]
    FP = df.agg(sum("FP")).collect()[0][0]
    FN = df.agg(sum("FN")).collect()[0][0]
    TN = df.agg(sum("TN")).collect()[0][0]
    precision = TP / (TP + FP)
    recall = TP / (TP + FN)
    f1_score = 2 * (precision * recall) / (precision + recall)
    accuracy = (TP + TN) / (TP + TN + FP + FN)
    precision = TP / (TP + FP)
    recall = TP / (TP + FN)
    return f1_score, accuracy, precision, recall

In [26]:
scores = get_scores(predictions)

ZeroDivisionError: division by zero

In [ ]:
print("F1 score: ", scores[0], "\nAccuracy: ", scores[1], "\nPrecision: ", scores[2], "\nRecall: ", scores[3])

Par les scores obtenus, on peut dire que le modèle n'est pas très performant. Cela est dû au fait que les données sont très déséquilibrées. En effet, il y a beaucoup plus de ratings négatifs que de ratings positifs. Il faudrait donc équilibrer les données pour avoir un modèle plus performant.
Par ailleurs, on aurait pu changer la méthode pour noter les ratings. En effet, on a choisi de noter les ratings en fonction de la distance entre le client et le vendeur. On aurait pu choisir de noter les ratings en fonction du temps de livraison, de la qualité de la nourriture, etc.

In [ ]:
# make a data frame with CID X LOC_NUM X VENDOR, and target (0,1)
# target = 1 if vendor is in the top 20 recommendations

# get top 20 recommendations for each user
top_20 = model.recommendForAllUsers(20)

# explode the recommendations
top_20 = top_20.withColumn("recommendations", explode("recommendations"))

# get the vendor_id from the recommendations
top_20 = top_20.withColumn("vendor_id", top_20["recommendations"]["vendor_index"])

# get the rating from the recommendations
top_20 = top_20.withColumn("rating", top_20["recommendations"]["rating"])

# drop the recommendations column
top_20 = top_20.drop("recommendations")

Pour ce qui est de la création du dataframe avec les targets, on a choisi de prendre les 20 premières recommandations pour chaque utilisateur. On a ensuite créé une colonne target qui vaut 1 si le vendeur est dans les 20 premières recommandations et 0 sinon.

In [ ]:
from pyspark.sql.types import IntegerType

# create a list of top 20 vendors for each user
top_20 = top_20.groupBy("CID X LOC_NUM X VENDOR_index").agg(collect_list("vendor_id").alias("top_20"))
top_20 = top_20.withColumn("top_20", top_20["top_20"].cast(ArrayType(IntegerType())))
top_20.show()

In [ ]:
# create column target on df_test with value 1 if vendor is in the top 20 recommendations
df_test = df_test.join(top_20, "CID X LOC_NUM X VENDOR_index", "left")
df_test = df_test.withColumn("target", when(array_contains("top_20", col("vendor_index")), 1).otherwise(0))

In [ ]:
# create a data frame with CID X LOC_NUM X VENDOR and target and drop duplicates
df_test = df_test.select("CID X LOC_NUM X VENDOR", "target").dropDuplicates()

In [ ]:
# create a submission.csv file with the target
df_submission = df_test.toPandas()
df_submission.info()
df_submission.sort_values(by=['CID X LOC_NUM X VENDOR'], inplace=True)
df_submission.to_csv('submission.csv', index=False)